This code is not commercial and created only for study purpose. The main aim is introduce users of  TU Dresden HPC-DA system with the TensorFlow and Jupyternotebook.
This code do not pretend on completeness or research significance.

Load core module with architectural components such layers. Load pretrained model 'Resnet' with 101 layers

In [ ]:
import torch
from torchvision import models
from torchvision import transforms



resnet = models.resnet101(pretrained=True)
torch.version.__version__

Pre-processing of images is a essential step for neural networks. The torchvision module provides pipeline of basic pre-processing functions. 

In [ ]:
from torchvision import transforms
preprocess = transforms.Compose([
transforms.Resize(256),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize(
mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225]
)])

Download and open image with the Pillow and python. This image from the open source educational EduPic library. You can try th with other images. Please respect copyright.

In [ ]:
from PIL import Image
import requests
import sys

url = 'http://www.edupic.net/Images/Reptiles/green_anole286a.JPG'

try:
    resp = requests.get(url, stream=True).raw

except requests.exceptions.RequestException as e:  
    sys.exit(1)

try:
    img = Image.open(resp)

except IOError:
    print("Unable to open image")
    sys.exit(1)

img.save('sid.jpg', 'jpeg')
img = Image.open('sid.jpg')
img

Pass the image through the pre-processing pipeline

In [ ]:
img_t = preprocess(img)
import torch
batch_t = torch.unsqueeze(img_t, 0)

In order to do inference (running a trained model on new data), we need to put the network in 'eval' mode.

In [ ]:
resnet.eval()

Ready to infrence

In [ ]:
out = resnet(batch_t)

We now need to find out what the label of the class that received the highest score. Load a text file listing the labels presented to the network during the training. This file containing the 1000 labels for the ImageNet dataset classes. We preloaded it already. You can download it from the zih cloudstore with the following code

In [ ]:
import requests
 
print('Download Starting...')
 
url = 'https://doc.zih.tu-dresden.de/hpc-wiki/pub/Compendium/PyTorch/imagenet_classes.txt'
 
r = requests.get(url)
 
filename = url.split('/')[-1] # this will take only -1 splitted part of the url
 
with open(filename,'wb') as output_file:
    output_file.write(r.content)
 
print('Download Completed')

At this point we need to find out the index corresponding to the maximum score in the out tensor we obtained above. Use the max fuction in PyTorch. It saves max values as the tensor  as well as the indicies where that maximum value occured  

In [ ]:
with open(filename) as f:
 labels = [line.strip() for line in f.readlines()]
 _, index = torch.max(out, 1)


First normalize outputs. Now we can use the index to acess the label. Index is a tensor here. So we need to get the actual numerical value to use as an index into labels list using index[0]. 

In [ ]:
 percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
 labels[index[0]], percentage[index[0]].item()


Use the sort function which sorts the values and provides the indicies. After that we finally can see top 5 best results.

In [ ]:
_, indices = torch.sort(out, descending=True)
best5 = [(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]
best5

Is it correct guess? :)